# Time

## On the philosophy of time

Starsim's implementation of time is complex, because, well, time is complex. Some questions we've had to wrestle with:

1. If you run a simulation from 2000 to 2020 with monthly timesteps, would you expect the same average result (say, number of infections) in 2004 (leap year) and 2005 (not a leap year)? What about February 2012 vs. March 2012?
2. Is a year 365 days, 365.25 days, or either 365 or 366 days depending on the year?
3. Usually years and dates are interchangeable, e.g. `float(2000) == ss.date('2000-01-01')`. So `start=2000, stop=2020` is equivalent to `start='2000-01-01', stop='2020-01-01'`. But what if someone wants to simply run for a number of years, e.g. `start=0, stop=20`, given that there is no year 0?
4. If the user has scheduled an event to happen at fractional year `2025.7`, do they mean the nearest day (`2025-09-12`) or do they mean that exact timestamp (`2025-09-11 16:48:00`)? Will users ever want to worry about hours and minutes? What about microseconds and nanoseconds?
5. Should units be strict, e.g. `ss.days(5) != 5`, or permissive, e.g. `ss.days(5) == 5`?
6. If a mortality rate is specified as "100 deaths per 1000 people per year", at the end of a year, should 100 people die ("100 deaths per 1000 people counted from the beginning to the end of a year"), or should 95 die ("100 deaths per 1000 **person**-years", since people stop accruing person-years after they die)?
7. What does it mean to multiply a probability? Is `ss.prob(0.1)*2 == ss.prob(0.2)`? If so, what is `ss.prob(0.5)*2` or `ss.prob(0.9)*2`?

Why are we telling you all this? Because although time seems intuitive, there are many corner cases where intuition breaks down (and, worse, it might not seem like it has broken down, leading you astray!). Thus, perhaps more than any other part of Starsim, be sure to *check your assumptions* about time. For example, in answer to the questions above (which are some but not all of the thorny issues we encountered!), we've made the following decisions – but we recognize that different decisions would also be valid:

1. A month is defined as exactly 1/12th of a year, so there are the same number of infections in 2004 and 2005, and in February vs. March.
2. A year is equal to 365 days (although some stretching happens to make integer years and calendar dates line up where needed).
3. Because there is no way to represent year 0 with a date object (e.g. `ss.date()`, which is based on `pd.Timestamp`), for this (important) special case we need to switch to a different object, `ss.datedur()` (more on that below).
4. We assume the user means integer days, unless they have _explicitly_ requested otherwise (e.g. `dt=ss.days(0.1)`).
5. Units are loose when comparing to unitless quantities (`ss.days(5) == 5`), but are strict when comparing with other units (e.g. `ss.years(1) == ss.days(365)`).
6. In Starsim v2, we chose the former by default. In Starsim v3, we decided we were wrong to do that and now choose the latter.
7. Probabilities are always constrained to `[0, 1]`, so `ss.prob(0.5)*2 == ss.prob(0.75)`, but this also means that `ss.prob(0.1)*2 ≈ ss.prob(0.19)`.

*More information about the philosophy of time is available [here](https://archive.org/details/being-and-time-martin-heidegger-1962).*


## Dates

Starsim defines a custom date object, `ss.date`, which is based on `pd.Timestamp`. It is quite flexible in terms of input:


In [ ]:
import pandas as pd
import datetime as dt
import starsim as ss

d1 = ss.date(2020)
d2 = ss.date(2020, 1, 1)
d3 = ss.date('2020-01-01')
d4 = ss.date(pd.Timestamp('2020-01-01'))
d5 = ss.date(dt.datetime(2020, 1, 1))

assert d1 == d2 == d3 == d4 == d5
print(d1)

Dates are interchangeable with floating point years (although be careful, as despite our best efforts, some rounding errors can still occur):

In [ ]:
d = ss.date('2025.08.02')
print(d)
print(d.years)
print(float(d)) # Alias to d.years

Like many Starsim objects, you can get all the properties of an `ss.date` object with `.disp()`:

In [ ]:
d.disp()

You can do some kinds of arithmetic on date, mostly addition or subtraction:

In [ ]:
d = ss.date('2015.8.10')
print(d)
print(d + ss.years(10))
print(d + ss.datedur(months=3, days=-5))

There is also an `ss.DateArray()` object, which you are unlikely to need to work with directly, but which is used to handle timelines (more on those later):

In [ ]:
import numpy as np
datearr = ss.date.from_array(np.arange(1990, 2020))
print('From an array:\n', datearr)

weekarr = ss.date.arange(start='2025-07-01', stop='2025-09-18', step=ss.week)
print('Using arange:\n', weekarr)

## Timepars

Dates say _when_ things happen, but disease modeling is really mostly about durations and rates: how may days from infection to recovery? What is the probability of transmission per week? In Starsim, we call these unit-aware quantities *time parameters* (or timepars for short), represented by the class `ss.TimePar`. Timepars are all available in four base units: days, weeks, months, or years (noting that a week is defined as exactly 7 days, and a month is defined as exactly 1/12th of a year). The full class hierarchy of the time parameters is:

```
TimePar  # All time parameters
├── dur  # All durations, units of *time*
│   ├── days  # Duration with units of days
│   ├── weeks
│   ├── months
│   ├── years
│   └── datedur  # Calendar durations
└── Rate  # All rates, units of *per* (e.g. per time or per event)
    ├── per  # Probability rates over time (e.g., death rate per year)
    │   ├── perday
    │   ├── perweek
    │   ├── permonth
    │   └── peryear
    ├── prob  # Unitless probability (e.g., probability of death per infection)
    │   ├── probperday
    │   ├── probperweek
    │   ├── probpermonth
    │   └── probperyear
    └── freq  # Number of events (e.g., number of acts per year)
        ├── freqperday
        ├── freqperweek
        ├── freqpermonth
        └── freqperyear
```


### Durations

Durations are relatively straightforward. The base class for durations is `ss.dur()`, but in almost all cases you'll want to use `ss.days()`, `ss.weeks()`, `ss.months()`, or `ss.years()` instead. (In fact, if you type `ss.dur(3, 'years')`, it will return `ss.years(3)`.)

Durations operate more or less how you would expect, with the left hand operator taking precedence:


In [ ]:
d1 = ss.years(2)
d2 = ss.days(3)
print(d1 + d2)
print(d2 + d1)

You can also easily convert durations:

In [ ]:
d1 = ss.years(10)
print(d1.days) # Represent as days, but keep as years internally
d2 = d1.to('days')
print(d2)

Note: in almost all cases, you can use either with or without the 's', i.e. `ss.years(1).to('day')` or `ss.years(1).to('days')` both work.

Also, there are shortcut classes for unit values:

In [ ]:
print(ss.weeks(1))
print(ss.week)
assert ss.weeks(1.0) == ss.weeks(1) == ss.week

#### Date durations

In most cases, the durations above can be used for arithmetic with dates. But sometimes we need precise date arithmetic that keeps track of each unit separately. For this we can use `ss.datedur`. For example:

In [ ]:
d = ss.date('2025.1.1')
years = ss.years(0.5)
datedur = ss.datedur(months=6)

print(d + years)
print(d + datedur)

Why is this happening? Because 2025.5 is closer to July 2nd than July 1st. There are fewer days from the start of the year to July 1st than there are days from July 1st to the end of the year:

In [ ]:
print(ss.date('2025.7.1') - ss.date('2025.1.1'))
print(ss.date('2025.12.31') - ss.date('2025.7.1'))

So even though a month is "usually" 1/12th of a year exactly, and although 6/12ths is certainly 0.5, this is an example where we get a different answer depending on whether we do date-based arithmetic or float-based arithmetic.


### Rates

#### Events vs. probabilities

TBC

## Timeline


In [ ]:
"""
Sanity check of TimeProb vs InstProb
"""
import numpy as np

# Probability of an event happening over a year -- "TimeProb"
timeprob = 0.1

# "InstProb" -- "Instantaneous probability per year" (which makes no sense)
instprob = -np.log(1-timeprob)

year = 365
iprob_per_day = timeprob/year # Convert to instantaneous probability per day
did_not_happen = 1
for day in range(year):
    did_not_happen *= 1-iprob_per_day
happened = 1 - did_not_happen

print(f'{timeprob = }')
print(f'{instprob = }')
print(f'{happened*1000 = }')
# assert np.isclose(happened, timeprob, atol=1e-3)